In [1]:
""" 
1. Использовать dataset MovieLens
2. Построить рекомендации ( предсказываем оценку)
на фичах:
■ TF IDF на тегах и жанрах
■ Средние оценки median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке
"""
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm  #  интеллектуальный индикатор выполнения - 
                                # просто оберните любую итерацию с помощью tqdm (iterable)

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline




In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [ ]:
# задача - мы должны понять, какую оценку пользователь поставит фильму (целевая переменная - rating)
# 1. Для этого мы изучим: 
#     1.1. фильмам каких жанров человек ставит оценки
#           (отфильтровать по пользователю, выделить фильмы, которым он поставил оценки, узнать жанры этих фильмов)
#     1.2. какие оценки ставит пользователь данным жанрам
#     1.3. как он размечает фильмы тегами 
#     1.4. расставим веса жанров и тегов методом tfidf 
# 2. соединяем таблицы фильмов и рейтингов, чтобы понимать, какому фильм какая оценка поставлена (для тестовой выборки)
# 3. сформируем линейную регрессию и сделаем прогноз
# 4. найдем ошибку RMSE

In [6]:
# тегов много: заменим редкие на постоянное значение 
tags.tag.nunique()

1589

In [7]:
# Заменим низкочастотные теги на значение other 

# Сделаем это так:
# Создадим отдельный Series-объект (назовем vc) с value_counts
# Отберем из него элементы с нужной частотой
# Используем индекс этих элементов - именно в индексе будет список нужных тегов (а в значениях там - число повторений тегов)
# Отфильтруем колонку с тегами по вхождению элементов столбца в список выше найденных редких тегов
# Присвоим нафильтрованным строкам в колонку с тегами наш новый тег

vc = tags['tag'].value_counts()
tags.loc[tags['tag'].isin(vc[vc < 10].index), 'tag'] = 'other'

In [8]:
# Проверим
rslt_df = tags.loc[tags['tag'] == 'other']
rslt_df.head()

,userId,movieId,tag,timestamp
1,2,60756,other,1445714996
2,2,60756,other,1445714992
3,2,89774,other,1445715207
4,2,89774,other,1445715200
5,2,89774,other,1445715205


In [9]:
# проверим количество уникальных значений тегов теперь; 
# при необходимости откалибруем фильтр частотности 2-мя ячейками выше (меняя частоту повторяемости 1500 в любую сторону)
tags.tag.nunique()

60

In [ ]:
# Обобщим возможности для экономии пространства:
# 1) заменить все редкие теги одним значением (например, other) можно разными способами:

# DataFrame['column_name'] = numpy.where(condition, new_value, DataFrame.column_name)
# еще можно так:
# movies_tags_ratings['tag'].replace(movies_tags_ratings['tag'].value_counts()<100, 'other')

# 2) параметрами ограничить число фич, создаваемых нашими трансформерами
# 3) мы оставили матрицу разреженной. В sklearn есть модели, которые умеют с ними работать, например, Naive Bayes
# 4) предварительно можно понизить размерность через TruncatedSVD, PCA

# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# pca = TruncatedSVD(n_components = 1)
# XPCAreduced = pca.fit_transform(transpose(movies_tags_ratings))

In [10]:
# некоторе теги встречаются заметно чаще остальных: это значит, что метод tfidf способен разнести веса от высоких к низким
# и тем не менее, вектор тегов слишком велик (соответствует всем тегам для каждого фильма + нулевые отметки, если 
# данный тег фильму не выставлялся)
tags['tag'].value_counts()

other                 2698
In Netflix queue       131
atmospheric             36
thought-provoking       24
superhero               24
funny                   23
surreal                 23
Disney                  23
religion                22
psychology              21
sci-fi                  21
dark comedy             21
quirky                  21
suspense                20
twist ending            19
crime                   19
visually appealing      19
politics                18
time travel             16
music                   16
mental illness          16
comedy                  15
aliens                  15
dark                    15
mindfuck                14
space                   14
dreamlike               14
black comedy            13
emotional               13
heist                   13
Stephen King            12
disturbing              12
Shakespeare             12
journalism              12
court                   12
anime                   12
satire                  12
a

In [11]:
# # Проверим
# rslt_df_leo = tags.loc[tags['movieId'] == '60756']
# # rslt_df_leo = tags.loc[tags['tag'] == 'Leonardo DiCaprio']
# rslt_df_leo.head()

# посмотрим, есть ли пустые значения столбца тегов: нет 
tags['tag'].isnull().any()

False

In [12]:
# Объявим функцию для работы с тегами
def change_tag_string(f):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце пробелом, 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(f.replace(' ', '').replace('-', '').split(' '))

In [13]:
# прогоним через функцию строки столбца тегов 
film_tags = [change_tag_string(g) for g in tags.tag.values]
tags['Movie_tags'] = film_tags
tags.sample(15)

,userId,movieId,tag,timestamp,Movie_tags
225,62,71535,other,1529777192,other
2436,474,31923,In Netflix queue,1137200865,InNetflixqueue
199,62,59501,other,1525637603,other
3517,599,296,other,1498456597,other
1436,474,1393,other,1137205489,other
3633,599,2959,other,1498456965,other
2034,474,5791,other,1138039181,other
2063,474,6027,In Netflix queue,1137200848,InNetflixqueue
1660,474,2660,aliens,1137521194,aliens
3452,599,296,other,1498456372,other


In [14]:
# все теги в каждой группе соединим через пробел
tags_group = tags.groupby('movieId', as_index=False).agg({'Movie_tags' : lambda x: ' '.join(set(x))})
tags_group.sample(15)

,movieId,Movie_tags
917,5528,other
972,6183,other
738,3566,religion other
573,2529,twistending
757,3859,InNetflixqueue other
1554,164179,thoughtprovoking cinematography visuallyappeal...
170,778,darkcomedy other
612,2750,other
231,968,other
1116,7069,InNetflixqueue Shakespeare


In [15]:
tags_group.shape

(1572, 2)

In [16]:
# Объявим функцию для работы с жанрами 
def change_string(s):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце символом '|', 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [17]:
# прогоним через функцию строки столбца жанров 
movie_genres = [change_string(g) for g in movies.genres.values]

# объявим новый столбец Movie_genres
movies['Movie_genres'] = movie_genres

In [18]:
# при помощи pd.merge соединим колонки с жанрами и тегами 
# (столбец genres заменим на модернизированный Movie_genres)
genres_tags = movies.merge(tags_group, on='movieId').drop(['genres'], axis=1)
genres_tags

,movieId,title,Movie_genres,Movie_tags
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,other
1,2,Jumanji (1995),Adventure Children Fantasy,other
2,3,Grumpier Old Men (1995),Comedy Romance,other
3,5,Father of the Bride Part II (1995),Comedy,remake other
4,7,Sabrina (1995),Comedy Romance,remake
...,...,...,...,...
1567,183611,Game Night (2018),Action Comedy Crime Horror,other funny
1568,184471,Tomb Raider (2018),Action Adventure Fantasy,other
1569,187593,Deadpool 2 (2018),Action Comedy SciFi,other
1570,187595,Solo: A Star Wars Story (2018),Action Adventure Children SciFi,other


In [19]:
"""
Далее применяются методы векторизации жанров и тегов для всего фрейма, обработки векторов методом tfidf
Будем рассматривать это отработкой подходов, которые в последующем применим к датафрейму конкретного пользователя
Модель предсказаний такова: мы через линейную регрессию получаем предсказания для фильмов:
их пользователь не смотрел, но его потенциальные оценки мы спрогнозировали
Логику рекомендательной системы можно сложить следующим образом:
получив наибольшие прогнозные оценки пользователя для картин, которые он еще не видел, 
мы будем предлагать ему эти фильмы к просмотру
"""
# Начнем отрабатывать применение моделей для всего датафрейма
# обучим модель для понимания, каким жанром принадлежит каждый их фильмов (получим вектор с жанрами для каждого фильма)
count_vect = CountVectorizer()
X_train_counts_genres = count_vect.fit_transform(genres_tags['Movie_genres'])
X_train_counts_genres.toarray()

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0]], dtype=int64)

In [20]:
# обучим модель, чтобы придать жанрам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf_gen = tfidf_transformer.fit_transform(X_train_counts_genres)

# реализовав метод, видим, что вес фильмов изменился в зависимости от его жанров (чем реже жанр, тем выше вес) 
X_train_tfidf_gen.toarray()

array([[0.        , 0.39861329, 0.52164113, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.49508056, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.59650626, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.44396322, 0.4561219 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.46534105, 0.        , 0.62564122, ..., 0.        , 0.        ,
        0.        ]])

In [21]:
# обучим модель для понимания, какими тегами описывается каждый из фильмов 
# для экономии памяти не будем трансформировать разреженную матрицу в обычный массив (.toarray())
# в итоге получаем вектор тегов для каждого фильма 
X_train_tags = count_vect.fit_transform(genres_tags['Movie_tags'])

In [22]:
# обучим модель, чтобы придать тегам различные веса 
X_train_tfidf_tags = tfidf_transformer.fit_transform(X_train_tags)

In [23]:
# объединим 2 массива колонок в одну матрицу 
X_train_gen_2 = X_train_tfidf_gen.todense()         # методом .todense() из разряженной матрицы сделали обычную из модуля numpy 
X_train_tfidf_tags_2 = X_train_tfidf_tags.todense()
hs = np.hstack( (X_train_gen_2, X_train_tfidf_tags_2) )

In [42]:
tags_movie_ = tags.merge(movies, on='movieId').drop(['genres', 'timestamp'], axis=1)
tags_movie_groupby = tags_movie_.groupby('movieId')
tags_movie_groupby.describe()
tags_movie_groupby.head()

,userId,movieId,tag,Movie_tags,title,Movie_genres
0,2,60756,funny,funny,Step Brothers (2008),Comedy
1,2,60756,other,other,Step Brothers (2008),Comedy
2,2,60756,other,other,Step Brothers (2008),Comedy
3,62,60756,comedy,comedy,Step Brothers (2008),Comedy
4,62,60756,funny,funny,Step Brothers (2008),Comedy
...,...,...,...,...,...,...
3678,606,5694,other,other,Staying Alive (1983),Comedy Drama Musical
3679,606,6107,other,other,Night of the Shooting Stars (Notte di San Lore...,Drama War
3680,606,7936,other,other,Shame (Skammen) (1968),Drama War
3681,610,3265,other,other,Hard-Boiled (Lat sau san taam) (1992),Action Crime Drama Thriller


In [32]:
movieId_as_series = tags_movie_groupby['movieId']
userId_as_matrix = tags_movie_groupby['userId']
shared_stack = np.hstack( (hs, movieId_as_series, userId_as_matrix) )

In [45]:
movieId_as_series.head()
userId_as_matrix.head()

0         2
1         2
2         2
3        62
4        62
       ... 
3678    606
3679    606
3680    606
3681    610
3682    610
Name: userId, Length: 2851, dtype: int64

In [33]:
shared_stack.shape

(1572, 84)

In [38]:
"""
Переходим к работе с конкретным пользователем. 
Применим все те методы, которые мы исследовали на примере всего фрейма данных
"""

# Выберем пользователя для обучения модели.
# Для этого посмотрим, количество тегов у фильмов, просмотренных данным пользователем. Выберем ТОП-10 пользователей.
# Пусть нашим пользователем станет user 537
user_data = tags_movie_.groupby('userId')['Movie_tags'].count().sort_values(ascending=False).head(10)
user_data

userId
474    1507
567     432
62      370
599     323
477     280
424     273
537     100
125      48
357      45
318      41
Name: Movie_tags, dtype: int64

In [78]:
# сформируем фрейм для конкретного пользователя для наглядности 
search_user = 537
user_films = (tags_movie_.loc[tags_movie_.userId == search_user]
        .merge(ratings, on='userId')
        .drop(['movieId_y', 'timestamp', 'tag'], axis=1))
#         .rename(columns={'movieId_x': 'movieId'}, inplace=True))
print(user_films.shape)
user_films.sample(15)

(4700, 6)


,userId,movieId_x,Movie_tags,title,Movie_genres,rating
1689,537,5989,other,Catch Me If You Can (2002),Crime Drama,5.0
4135,537,97304,other,Argo (2012),Drama Thriller,2.5
4651,537,105504,other,Captain Phillips (2013),Adventure Drama Thriller IMAX,5.0
1693,537,72998,other,Avatar (2009),Action Adventure SciFi IMAX,4.0
1724,537,72998,other,Avatar (2009),Action Adventure SciFi IMAX,5.0
1453,537,5989,other,Catch Me If You Can (2002),Crime Drama,3.0
2126,537,8533,other,"Notebook, The (2004)",Drama Romance,5.0
904,537,79132,other,Inception (2010),Action Crime Drama Mystery SciFi Thriller IMAX,5.0
4318,537,104841,other,Gravity (2013),Action SciFi IMAX,0.5
1377,537,5989,other,Catch Me If You Can (2002),Crime Drama,4.5


In [79]:
# реализуем для датафрейма user_films метод tf-idf для жанров
X_count_genres_user_537 = count_vect.fit_transform(user_films['Movie_genres'])
# придадим вектору жанров различные веса 
X_tfidf_genres_user_537 = tfidf_transformer.fit_transform(X_count_genres_user_537)
print('Размерность матрицы жанров: ', X_tfidf_genres_user_537.shape)

Размерность матрицы жанров:  (4700, 14)


In [80]:
# реализуем для датафрейма user_films метод tf-idf для тегов 
X_count_tags_user_537 = count_vect.fit_transform(user_films['Movie_tags'])
# придадим вектору тегов различные веса 
X_tfidf_tags_user_537 = tfidf_transformer.fit_transform(X_count_tags_user_537)
print('Размерность матрицы тегов: ', X_tfidf_tags_user_537.shape)

Размерность матрицы тегов:  (4700, 17)


In [85]:
# методом .todense() из разряженной матрицы сделаем обычную из модуля numpy
X_gen_user537 = X_tfidf_genres_user_537.todense()         
X_tag_user537 = X_tfidf_tags_user_537.todense()

In [101]:
# соединим полученные векторы
genres_tags_together = np.hstack( (X_gen_user537, X_tag_user537) )

In [103]:
# уберем из фрейма пользователя столбцы, которые будут излишними при обучении линейной регрессии:
# id пользователя, id фильма и название фильма
user_films_537 = pd.DataFrame(genres_tags_stack)
# user_films_537 = genres_tags_stack.drop(['userId', 'movieId_x', 'title'], axis=1)

In [106]:
# займемся обучением линейной регрессии. Для начала объявим переменные X и y
X = genres_tags_together
y= user_films['rating']

In [107]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [108]:
# посмотрим на качество модели. На мой взгляд, оно аномальное 
from sklearn.metrics import mean_squared_error as mse
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

mse(y_train, y_train_pred), mse(y_test, y_test_pred)

(1.3183504567862947, 1.423120690982324)

In [ ]:
# Получим средние оценки median, variance, etc.) по фильмам
movies_and_ratings = movies.merge(ratings, on='movieId')
movies_stat_film = movies_and_ratings.groupby('title')['rating'].agg([lambda x: np.mean(x), lambda x: np.median(x), lambda x: np.var(x), lambda x: np.std(x)])
movies_stat_film.columns = ['mean', 'median', 'variance', 'std']
movies_stat_film.sample(10)

In [ ]:
# Получим средние оценки median, variance, etc.) по пользователям

movies_stat_user = movies_and_ratings.groupby('userId')['rating'].agg([lambda x: np.mean(x), lambda x: np.median(x), lambda x: np.var(x), lambda x: np.std(x)])
movies_stat_user.columns = ['mean', 'median', 'variance', 'std']
movies_stat_user.sample(10)